In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")
    
import sumolib

In [2]:
import pickle

with open('data/small_features.pkl', 'rb') as f:
    features = pickle.load(f)

with open('data/small_speeds.pkl', 'rb') as f:
    speeds = pickle.load(f)
    
with open('data/small_coords.pkl', 'rb') as f:
    coords = pickle.load(f) 

In [3]:
import pandas as pd

edges = pd.read_csv("data/small.csv", names=["source", "dest"])
edges.head()

,source,dest
0,1614922744,1614922747
1,269459339,269459341
2,1621320618,1621320622
3,269459229,269459230
4,607954833,1613371666


In [4]:
features

{('1614922744',
  '1614922747'): array([  34.04259625, -118.22076005,    2.        ,   65.        ,
          23.14193894,    0.        ,    1.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ]),
 ('269459339',
  '269459341'): array([  34.05863   , -118.21434725,    5.        ,   65.        ,
          60.826398  ,    1.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ]),
 ('1621320618',
  '1621320622'): array([  34.05482275, -118.2000536 ,    2.        ,   65.        ,
         109.46496614,    0.        ,    1.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ]),
 ('269459229',
  '269459230'): array([  34.0547308 , -118.2109487 ,    3.        ,  

In [7]:
coords

{'360710162': (-117.482101, 33.3171985),
 '360710163': (-117.4821557, 33.3178217),
 '360710196': (-117.4824534, 33.3175213),
 '360710206': (-117.4825237, 33.3182306),
 '243269768': (-122.2974056, 37.8210286),
 '252357330': (-117.2388816, 32.925212),
 '6551503108': (-118.8413169, 34.168127),
 '92261869': (-122.378534, 41.054464),
 '497025082': (-117.6878708, 33.9666775),
 '497025084': (-117.6876277, 33.9662538),
 '1595233685': (-118.4549285, 34.2785361),
 '5244267931': (-122.3385237, 40.5788174),
 '497025093': (-117.6909063, 33.9670531),
 '708138396': (-122.0695999, 37.4000926),
 '252357376': (-117.2382651, 32.9233919),
 '4074067389': (-118.6249933, 34.1637649),
 '280319424': (-118.3226186, 34.1041326),
 '1396703911': (-117.3307788, 33.1810213),
 '243270389': (-122.2969067, 37.8203421),
 '243270391': (-122.296727, 37.8198273),
 '625976984': (-118.4041271, 33.9938927),
 '20972360': (-118.2291472, 34.0850589),
 '20972369': (-118.2395618, 34.0552723),
 '20972370': (-118.2421453, 34.0568794

In [8]:
# Create node XML
import xml.etree.cElementTree as ET

nodes = ET.Element("nodes")
for n in set(edges["source"]).union(set(edges["dest"])):
    v = str(n)
    long, lat = coords[v]
    node = ET.SubElement(nodes, "node", id=v, x=str(long), y=str(lat))
    
tree = ET.ElementTree(nodes)
tree.write("out/small.nodes.xml")

In [9]:
# Create edge XML

edgeXML = ET.Element("edges")
i = 0
for i, e in enumerate(edges.itertuples()):
    feat = features[tuple([str(v) for v in e[1:3]])]
    numLanes = str(int(feat[2]))
    ET.SubElement(edgeXML, "edge", {"id": str(i), "from": str(e[1]), "to": str(e[2]), 
                                    "numLanes": str(int(feat[2])), "speed": str(feat[3] * 0.44704), "length": str(feat[4])})
    
tree = ET.ElementTree(edgeXML)
tree.write("out/small.edges.xml")

In [14]:
# Run netconvert first

import numpy as np
from tqdm.notebook import tqdm

net = sumolib.net.readNet('out/small.net.xml')

with open('data/small_flows.pkl', 'rb') as f:
    flows = pickle.load(f)

readings = pd.DataFrame(
    {
        "Detector": [],
        "Time": [],
        "qPKW": [],
        "vPKW": []
    }
)

detectors = ET.Element("detectors")

for i, e in tqdm(enumerate(net.getEdges())):
    key = (str(e.getFromNode().getID()), str(e.getToNode().getID()))
    if key in speeds.keys() and key in flows.keys():
        # Add detectors to XML
        for lane in e.getLanes():
            ET.SubElement(detectors, "detectorDefinition", id="det" + str(lane.getID()), 
                          lane=str(lane.getID()), pos=str(lane.getLength() / 2))
        
            # Add info to csv
            speed = speeds[key] * 0.44704 # Convert from mph to m/s
            flow = flows[key] / (len(e.getLanes())) # Convert from hourly average to minute average
            for j in range(24):
                readings.loc[len(readings.index)] = ["det" + str(lane.getID()), 60 * j, int(flow), speed] 
        
tree = ET.ElementTree(detectors)
tree.write("out/small.detectors.xml")

readings.head()

0it [00:00, ?it/s]

,Detector,Time,qPKW,vPKW
0,det1046_0,0.0,833.0,29.307504
1,det1046_0,60.0,833.0,29.307504
2,det1046_0,120.0,833.0,29.307504
3,det1046_0,180.0,833.0,29.307504
4,det1046_0,240.0,833.0,29.307504


In [15]:
readings.to_csv("out/small_readings.csv", sep=";", index=False)

In [ ]:
import matplotlib.pyplot as plt

x, y = zip(*coords.values())

plt.hist(x)
plt.show()

In [ ]:
features

In [ ]:
flows.keys()